In [10]:
import os
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

img_width, img_height = 48, 48  # dimensions to which the images will be resized
batch_size = 32  # number of images per batch
epochs = 50  # number of epochs to train for

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))  # Updated input shape
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))  # Removed groups parameter
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

train_data_dir = 'C:/Users/sides/Downloads/data/train'
validation_data_dir = 'C:/Users/sides/Downloads/data/test'

nb_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)])
nb_validation_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])

train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb')  # Specifying color mode as RGB
                                                                    
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb')  # Specifying color mode as RGB

model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/50
897/897 [==============================] - 297s 329ms/step - loss: 1.7550 - accuracy: 0.2841 - val_loss: 1.6593 - val_accuracy: 0.3810
Epoch 2/50
897/897 [==============================] - 49s 54ms/step - loss: 1.6230 - accuracy: 0.3651 - val_loss: 1.4950 - val_accuracy: 0.4268
Epoch 3/50
897/897 [==============================] - 48s 53ms/step - loss: 1.5552 - accuracy: 0.3980 - val_loss: 1.3998 - val_accuracy: 0.4541
Epoch 4/50
897/897 [==============================] - 51s 56ms/step - loss: 1.5062 - accuracy: 0.4172 - val_loss: 1.3531 - val_accuracy: 0.4782
Epoch 5/50
897/897 [==============================] - 52s 58ms/step - loss: 1.4792 - accuracy: 0.4267 - val_loss: 1.3354 - val_accuracy: 0.4911
Epoch 6/50
897/897 [==============================] - 52s 58ms/step - loss: 1.4569 - accuracy: 0.4405 - val_loss: 1.3326 - val_accuracy: 0.4933
Epoch 7/50
897/897 [=============================

In [11]:
# After training
model.save('facial_expression_model.h5')

C:\Users\sides\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
from keras.models import load_model

# Load the model
loaded_model = load_model('facial_expression_model.h5')


In [23]:
from keras.preprocessing import image
import numpy as np
import os 
from keras.models import load_model

# Load the model
loaded_model = load_model('facial_expression_model.h5')

# Define a function to preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(48, 48))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

# Define a function to make predictions on the preprocessed image
def predict_image(image_path, model):
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    return prediction

# Test with a random image
image_path = "D:/SEM-6/NLP/Sticker/13.jpg"
prediction = predict_image(image_path, loaded_model)
print(prediction)


1/1 [==============================] - 0s 92ms/step
[[8.5411504e-02 1.0677345e-04 1.1229884e-01 3.5743147e-01 2.8809187e-01
  1.3690968e-01 1.9749822e-02]]


In [31]:
# Define the labels
labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Test with a random image
image_path = "D:/SEM-6/NLP/Sticker/18.jpg"
prediction = predict_image(image_path, loaded_model)

# Interpret the prediction
predicted_label = labels[np.argmax(prediction)]
print("Predicted expression:", predicted_label)


1/1 [==============================] - 0s 40ms/step
Predicted expression: Angry
